In [ ]:
from fuzzywuzzy import process, fuzz
import re
import matplotlib.pyplot as plt
import numpy as np
from rapidfuzz import fuzz
import os
import json
import string
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
from sentence_transformers import SentenceTransformer, util
import requests
from bs4 import BeautifulSoup
from io import StringIO

### Parsing prices data from web pages

In [ ]:
# Getting the html page
import requests

URL_base = "https://osservaprezzi.mise.gov.it/prezzi/livelli/beni-e-servizi-di-largo-consumo/archivio-rilevazioni-beni-e-servizi-di-largo-consumo?ANNO=2025&MESE=5&f%5BPROVINCIA%5D=Bologna&f%5BTIPO_RECORD_MISE%5D="
URL_alimenti = "altri_alim&submit=Applica"
URL_ortofrutta = "orto&submit=Applica"
URL_ittici = "ittici&submit=Applica"
page_alimenti = requests.get(URL_base+URL_alimenti)
page_ortofrutta = requests.get(URL_base+URL_ortofrutta)
page_ittici = requests.get(URL_base+URL_ittici)

print(page_alimenti.text)

In [ ]:
# parsing the page with BeautifulSoup
soup_al = BeautifulSoup(page_alimenti.content, "html.parser")
soup_ort = BeautifulSoup(page_ortofrutta.content, "html.parser")
soup_itt = BeautifulSoup(page_ittici.content, "html.parser")

In [ ]:
# finding the table with id "id-table-results"
results_alimenti = soup_al.find(id="id-table-results")
results_orto = soup_ort.find(id="id-table-results")
results_ittici = soup_itt.find(id="id-table-results")

In [ ]:
# Transform the table into a pandas dataframe
df_prices_alimenti = pd.read_html(StringIO(str(results_alimenti)))[0]
df_prices_orto = pd.read_html(StringIO(str(results_orto)))[0]
df_prices_ittici = pd.read_html(StringIO(str(results_ittici)))[0]

prices_data = pd.concat([df_prices_alimenti, df_prices_orto], ignore_index=True)
prices_data = pd.concat([prices_data, df_prices_ittici], ignore_index=True)

In [ ]:
prices_data

In [ ]:
# Initialize SPARQL endpoint and model
sparql = SPARQLWrapper("http://agrovoc.fao.org/sparql")
model = SentenceTransformer('all-MiniLM-L6-v2')

CACHE_PATH = "agro_cache.json"

In [ ]:
def load_cache():
    if not os.path.exists(CACHE_PATH):
        return {}
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        cache = json.load(f)
        return cache

def save_cache(cache):
    with open(CACHE_PATH, "w", encoding="utf-8") as f:
        json.dump(cache, f, indent=2)

def clean_cache():
    if not os.path.exists(CACHE_PATH):
        return
    with open(CACHE_PATH, "r", encoding="utf-8") as f:
        cache = json.load(f)

    cleaned_cache = {
        key: value
        for key, value in cache.items()
        if value.get("label") is not None
    }

    with open(CACHE_PATH, "w", encoding="utf-8") as f:
        json.dump(cleaned_cache, f, indent=2)

In [ ]:
def semantic_best_match(value, candidates):
    input_embedding = model.encode(value, convert_to_tensor=True)
    candidate_texts = []
    for c in candidates:
        candidate_texts.append(c["label"])
    candidate_embeddings = model.encode(candidate_texts, convert_to_tensor=True)
    scores = util.cos_sim(input_embedding, candidate_embeddings)[0]
    best_idx = scores.argmax().item()
    return candidates[best_idx]

In [ ]:
def query_agrovoc(value, cache):

    norm_value = value.lower().replace(",", "")

    if norm_value in cache:
        print(f"Cache hit: '{value}' found in cache")
        return cache[norm_value]
    
    for entry in cache.values():
        if "altLabels" in entry and value in entry["altLabels"]:
            print(f"Cache hit: '{value}' found in cache")
            return entry
    
    
    print(f"Cache miss: '{value}' not in cache, querying AGROVOC endpoint")
    # Run SPARQL query
    query = f"""
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

        SELECT ?concept ?label (GROUP_CONCAT(?altLabel; separator=" | ") AS ?altLabels) WHERE {{
        ?concept skos:prefLabel ?label .
        OPTIONAL {{
            ?concept skos:altLabel ?altLabel .
            FILTER(LANG(?altLabel) = "en")
        }}

        FILTER(LANG(?label) = "en")

        FILTER(
            CONTAINS(LCASE(?label), "{norm_value}") ||
            (BOUND(?altLabel) && CONTAINS(LCASE(?altLabel), "{norm_value}"))
        )
        }}
        GROUP BY ?concept ?label

    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    candidates = []
    for result in results["results"]["bindings"]:
        label = result["label"]["value"]
        uri = result["concept"]["value"]
        altlabels_str = result["altLabels"]["value"] if "altLabels" in result else ""
        altlabels = [al.strip().lower() for al in altlabels_str.split("|") if al.strip()]
        candidate = {"label": label, "uri": uri, "altLabels": altlabels}
        candidates.append(candidate)
        
    if not candidates:
        match = {"label": None, "uri": None, "altLabels": []}
    else:
        best = semantic_best_match(value, candidates)
        match = best
    # Cache the result
    cache[norm_value] = match
    save_cache(cache)
    return match

In [ ]:
def enrich_with_agrovoc(df, column_name):
    cache = load_cache()
    labels = []
    uris = []
    for val in df[column_name]:
        match = query_agrovoc(val, cache)
        labels.append(match["label"])
        uris.append(match["uri"])
    df["AGROVOC_label"] = labels
    df["AGROVOC_uri"] = uris
    return df

In [ ]:
# Load the food consumption data
df_consumption = pd.read_csv("data/chronic_consumption_gday_allsubjects.csv", encoding="utf-16")

# Load the SuEatableLife dataset
df_wf = pd.read_excel("data/sueatablelife_dataset.xlsx", sheet_name="SEL WF for users")
df_cf = pd.read_excel("data/sueatablelife_dataset.xlsx", sheet_name="SEL CF for users")

In [ ]:
# Remove water from the consumption data
df_consumption = df_consumption[(df_consumption["Exposure hierarchy (L7)"] != "Natural mineral water") &
    (df_consumption["Exposure hierarchy (L7)"] != "Tap water") &
    (df_consumption["Exposure hierarchy (L7)"] != "Filtered tap water")]

# Group by "Exposure hierarchy (L5)" and sum the "Mean" values
top_n = df_consumption.groupby("Exposure hierarchy (L5)", as_index=False)["Mean"].sum().sort_values('Mean', ascending=False).head(15)

top_n[["Exposure hierarchy (L5)", "Mean"]]

In [ ]:
# Merge coffee rows and milk rows
rows_to_merge = top_n[top_n["Exposure hierarchy (L5)"].isin(["Coffee (average strength) beverage", "Coffee espresso (beverage)"])]
merged_row = rows_to_merge.sum(numeric_only=True)
merged_row["Exposure hierarchy (L5)"] = "Coffee"

# Drop the original rows and append the merged row
top_n = top_n[~top_n["Exposure hierarchy (L5)"].isin(["Coffee (average strength) beverage", "Coffee espresso (beverage)"])]
top_n = pd.concat([top_n, pd.DataFrame([merged_row])], ignore_index=True)

In [ ]:
top_n

In [ ]:
# Sort again by "Mean" values to include the merged rows
top_n = top_n.sort_values("Mean", ascending=False).reset_index(drop=True)

top_n

In [ ]:
# Enrich the top_n DataFrame with AGROVOC labels and URIs
top_n_enriched = enrich_with_agrovoc(top_n, "Exposure hierarchy (L5)")

top_n_enriched

In [ ]:
# Functions for pre-processing
# Function to singularize English words based on common pluralization rules.
# List of (pattern, replacement) in priority order:
PLURAL_RULES = [
    # -kies → -kie  (e.g. “cookies” → “cookie”)
    (r'(?i)([a-z]+)kies$', r'\1kie'),
    # -ies → -y      (e.g. “berries” → “berry” but not “cookies”)
    (r'(?i)([a-z]+)ies$', r'\1y'),
    # -ves → -f      (e.g. “wolves” → “wolf”)
    (r'(?i)([a-z]+)ves$', r'\1f'),
    # -oes → -oe     (e.g. “heroes” → “heroe”—rare, you may want 'o')
    (r'(?i)([a-z]+)oes$', r'\1oe'),
    # -ses → -s      (e.g. “dresses” → “dress”)
    (r'(?i)([a-z]+)ses$', r'\1s'),
    # -xes → -x      (e.g. “boxes” → “box”)
    (r'(?i)([a-z]+)xes$', r'\1x'),
    # -ches/-shes → -ch/-sh  (e.g. “churches” → “church”)
    (r'(?i)([a-z]+(?:ch|sh))es$', r'\1'),
    # -s → ''        (catch‑all; e.g. “cars” → “car”)
    (r'(?i)([a-z]+)s$', r'\1'),
]

def singularize(word: str) -> str:
    """Apply common English plural→singular regex rules."""
    for pattern, repl in PLURAL_RULES:
        if re.search(pattern, word):
            return re.sub(pattern, repl, word)
    return word  # no rule matched

In [ ]:
# Function to normalize labels
def normalize_label(s, language='en'):
    s = s.lower().strip().replace('-', ' ').replace('_', ' ')
     # Remove numbers and letters in brackets, asteriks, punctuation
    s = re.sub(r'\(.*?\)', '', s)  # Remove text in brackets
    s = re.sub(r'\d+', '', s)  # Remove numbers
    s = re.sub(r'\*', '', s)  # Remove asterisks
    s = re.sub(r'[^a-z0-9 ]+', ' ', s)       # drop punctuation
    s = re.sub(r'\b(semi[- ]skimmed|organic|low[- ]fat|mixed)\b', '', s)
    s = re.sub(r'\s+', ' ', s)               # collapse whitespace
    s = re.sub(r"(fresh|caffeinic|common)", "", s) # remove common useless attributes
    s = re.sub(r"wheat bread", "bread", s)
    s = s.strip()  # Remove leading and trailing whitespace
    s = re.sub(r"meat", "", s) # remove meat to improve the results of fuzzy matching for different animal meat
    s = re.sub(r"\sor\s", " ", s)
    # handling plurals
    if language == 'en':
        s = singularize(s)

    return s

In [ ]:
# Normalize the AGROVOC labels of the first dataframe
top_n['clean_label'] = top_n["AGROVOC_label"].apply(normalize_label)

In [ ]:
top_n

In [ ]:
# Normalize the labels in the other dataframes
df_cf['clean_label'] = df_cf["Food commodity ITEM"].apply(normalize_label)
df_wf['clean_label'] = df_wf["Food commodity ITEM"].apply(normalize_label)

In [ ]:
# Function to create a mapping from a dataframe column to the closest values in another dataframe column using fuzzy matching
def create_fuzzy_mapping(source_df, target_df, source_col, target_col, score_cut):
    matches = {}
    for value in source_df[source_col]:
        result = process.extractOne(value, target_df[target_col], score_cutoff=score_cut)
        if result is not None:
            match, score, _ = result
            matches[value] = match
        # Keep unmatched values as None
        else:
            matches[value] = None
    return matches

In [ ]:
matches = create_fuzzy_mapping(top_n, df_cf, 'clean_label', 'clean_label', 85)

In [ ]:
# Add the matched food to the top_n DataFrame, but keep unmatched values as None
top_n['matched_food'] = top_n['clean_label'].map(matches)

In [ ]:
# Merge the top_n DataFrame with the cF DataFrame on the matched food, keeping unmatched values as None
merged_df = top_n.merge(df_cf, left_on='matched_food', right_on='clean_label', how="left", suffixes=('_cons', '_cf'))

In [ ]:
merged_df

In [ ]:
# Manually fix the data
# Search for coffee rows in the cF DataFrame

df_cf[df_cf["clean_label"].str.contains("coffe", case=False, na=False)]

In [ ]:
# Choose the correct coffee row in the cF DataFrame
single_coffee_row = df_cf[df_cf["clean_label"] == "coffee drip filtered"]

In [ ]:
# Function to substitute part of the rows in a DataFrame
# This function substitutes the last n columns of rows in df1 where the specified column matches value
def substitute_part_rows(df1, df2, value1, column_name, n_columns, df3=None, df4=None, value2=None, value3=None):
    for idx, row in df1.iterrows():
        if pd.isna(row[column_name]):
            continue
        if value1 in row[column_name].lower():
            df1.iloc[idx, -n_columns:] = df2.iloc[0,:n_columns].values
        if df3 is not None:
            if value2 in row[column_name].lower():
                df1.iloc[idx, -n_columns:] = df3.iloc[0,:n_columns].values
        if df4 is not None:
            if value3 in row[column_name].lower():
                df1.iloc[idx, -n_columns:] = df4.iloc[0,:n_columns].values

In [ ]:
""" # Substitute the values in the merged DataFrame for the coffee row
for idx, row in merged_df.iterrows():
    if row["Food commodity ITEM"] == "COFFEE GROUND":
        merged_df.iloc[idx, -10:] = single_coffee_row.iloc[0,:10].values
        break """

# Use the function to substitute the values in the merged DataFrame for the coffee row
substitute_part_rows(merged_df, single_coffee_row, "COFFEE", "Food commodity ITEM", 10)

In [ ]:
merged_df

In [ ]:
merged_df.drop(columns=["Exposure hierarchy (L5)", "Uncertainty    low (L) high (H)", "matched_food", "FOOD COMMODITY GROUP", "Food commodity sub-TYPOLOGY", "Carbon Footprint g CO2eq/g o cc of food sub-TYPOLOGY", "clean_label_cf"])

In [ ]:
matches = create_fuzzy_mapping(merged_df, df_wf, 'clean_label_cons', 'clean_label', 85)

In [ ]:
merged_df['matched_food_wf'] = merged_df['clean_label_cons'].map(matches)

In [ ]:
new_merged_df = merged_df.merge(df_wf, left_on='matched_food_wf', right_on='clean_label', how="left", suffixes=('_cons', '_wf'))

In [ ]:
new_merged_df

In [ ]:
# Remove dried apples (wrong match)
new_merged_df = new_merged_df[~(new_merged_df["Food commodity ITEM_wf"] == "APPLES")]

In [ ]:
# Manually fix the data
# Choose the correct coffee
df_wf[df_wf["clean_label"].str.contains("coffee", case=False, na=False)]

In [ ]:
single_coffee_row = df_wf[df_wf["clean_label"] == "coffee standard"]

In [ ]:
substitute_part_rows(new_merged_df, single_coffee_row, "COFFEE ROASTED", "Food commodity ITEM_wf", 10)

In [ ]:
# Apply preprocessing to the prices data
prices_data['clean_label'] = prices_data["Descrizione Prodotto"].apply(normalize_label, language='it')

prices_data

In [ ]:
import torch

def find_semantic_matches(source_df, candidate_df, source_column, candidate_column, model_name='paraphrase-multilingual-mpnet-base-v2'):
    """
    Finds the best semantic match for each item in a source DataFrame column from a candidate DataFrame column.

    Args:
        source_df (pd.DataFrame): The DataFrame containing the items to be matched.
        candidate_df (pd.DataFrame): The DataFrame containing the pool of candidate items.
        source_column (str): The name of the column in source_df to match from.
        candidate_column (str): The name of the column in candidate_df to match against.
        model_name (str): The name of the sentence-transformer model to use.

    Returns:
        pd.DataFrame: The source DataFrame with a new 'best_match' column.
    """
    # 1. Load the pre-trained multilingual model
    model = SentenceTransformer(model_name)

    # 2. Get the lists of strings to compare
    source_items = source_df[source_column]
    candidate_items = candidate_df[candidate_column]

    # 3. Encode the candidate strings into embeddings (done only once for efficiency)
    print("Encoding candidate embeddings...")
    candidate_embeddings = model.encode(candidate_items.tolist(), convert_to_tensor=True)
    print("Encoding complete.")

    # 4. Iterate through the source column to find the best match for each item
    best_matches = []
    print(f"Finding best matches for {len(source_items)} source items...")
    for item in source_items:
        # Encode the source item
        item_embedding = model.encode(item, convert_to_tensor=True)
        
        # Compute cosine similarity between the item and all candidates
        cosine_scores = util.cos_sim(item_embedding, candidate_embeddings)
        
        # Find the index of the highest score
        best_match_index = torch.argmax(cosine_scores).item()
        
        # Get the best matching string from the candidate list
        best_match_string = candidate_items.iloc[best_match_index]
        
        best_matches.append(best_match_string)
    
    print("Matching complete.")
    # 5. Add the results to the source DataFrame
    source_df['best_match'] = best_matches
    
    return source_df

# --- Example of how to use the function with your actual data ---
# Find the best price description for each food item in your merged dataframe
final_df = find_semantic_matches(
    source_df=new_merged_df,
    candidate_df=prices_data,
    source_column='clean_label_cons',
    candidate_column='clean_label'
)

# Display the key columns to check the results
final_df[['clean_label_cons', 'best_match']]

In [ ]:
# Merging the final DataFrame with prices data
final_prices_df = final_df.merge(prices_data, left_on='best_match', right_on='clean_label', how="left", suffixes=('_con', '_price'))

In [ ]:
final_prices_df

In [ ]:
# Manually fix the data
# Choose the correct food items
# For beef, pasta, and tomato, we will select the rows that match our criteria

beef_price_row = prices_data[prices_data["Descrizione Prodotto"].str.contains("bovino adulto", case=False, na=False)]
pasta_price_row = prices_data[prices_data["Descrizione Prodotto"].str.contains("pasta", case=False, na=False)]
tomato_price_row = prices_data[prices_data["Descrizione Prodotto"].str.contains("pomodori", case=False, na=False)]

tomato_price_row.shape

In [ ]:
# substitute the values in the final_prices_df
substitute_part_rows(final_prices_df, beef_price_row, "carne", "Descrizione Prodotto", 6, df3=pasta_price_row, value2="pasta", df4=tomato_price_row, value3="pomodoro")

""" for idx, row in final_prices_df.iterrows():
    if "carne" in row["Descrizione Prodotto"].lower():
        final_prices_df.iloc[idx, -6:] = beef_price_row.iloc[0,:6].values
    elif "pasta" in row["Descrizione Prodotto"].lower():
        final_prices_df.iloc[idx, -6:] = pasta_price_row.iloc[0,:6].values
    elif "pomodoro" in row["Descrizione Prodotto"].lower():
        final_prices_df.iloc[idx, -6:] = tomato_price_row.iloc[0,:6].values """

In [ ]:
final_prices_df

In [ ]:
# Manually fix the data for peaches
# Put NaN for last seven columns of the row with peaches

peach_row = final_prices_df[final_prices_df["clean_label_con"].str.contains("peach", case=False, na=False)]

for idx, row in peach_row.iterrows():
    final_prices_df.iloc[idx, -7:] = [np.nan] * 7  # Set the last seven columns to NaN

final_prices_df

In [ ]:
import numpy as np

# Helper function to determine the final CF and WF values
def get_final_values(row):
    cf_value = np.nan
    wf_value = np.nan
    
    if row["Suggested CF value"] in ["OK item", "Item matching typology"]:
        cf_value = row["Carbon Footprint kg CO2eq/kg or l of food ITEM"]
    elif row["Suggested CF value"] == "better subtypology":
        cf_value = row["Carbon Footprint g CO2eq/g o cc of food sub-TYPOLOGY"]
    elif row["Suggested CF value"] == "better typology":
        cf_value = row["Carbon Footprint g CO2eq/g o cc of food TYPOLOGY"]
    
    if row["Suggested WF value"] in ["ok item", "item matching typology"]:
        wf_value = row["Water Footprint liters water/kg o liter of food ITEM"]
    elif row["Suggested WF value"] == "better subtypology":
        wf_value = row["Water Footprint cc water/g o cc of food sub-TYPOLOGY"]
    elif row["Suggested WF value"] == "better typology":
        wf_value = row["Water Footprint cc water/g o cc of food TYPOLOGY"]
    
    return pd.Series([cf_value, wf_value])

# Apply the helper function to the DataFrame
final_prices_df[["Final CF value", "Final WF value"]] = final_prices_df.apply(get_final_values, axis=1)

# Display the final DataFrame with the new columns
final_prices_df[["Food commodity ITEM_cons", "Final CF value", "Food commodity ITEM_wf", "Final WF value"]]

In [ ]:
final_prices_df["Descrizione Prodotto"]

In [ ]:
# Function to extract the food quantity and unit from the description
def extract_food_quantity(description):
    # Use regex to find the quantity in the description
    if isinstance(description, str):
        match = re.search(r"(\d+)\s*([a-zA-Z]{2})", description, re.IGNORECASE)
        if match:
            quantity = match.group(1)
            unit = match.group(2).lower()
            return int(quantity), unit
    return None, None  # Return None if no match is found

In [ ]:
# Apply the function to extract food quantity and unit
final_prices_df['food_quantity'], final_prices_df['food_unit'] = zip(*final_prices_df['Descrizione Prodotto'].apply(extract_food_quantity))
# Display the final DataFrame with the new columns
final_prices_df[["Food commodity ITEM_cons", "Final CF value", "Food commodity ITEM_wf", "Final WF value", "food_quantity", "food_unit"]]

In [ ]:
# Function to calculate the cost per kg or liter
def calculate_cost_per_kg_or_liter(row):
    if row['food_unit'] in ['kg', 'l']:
        return row['Quotazione Media'] / row['food_quantity']
    elif row['food_unit'] in ['gr', 'ml']:
        return (row['Quotazione Media'] / row['food_quantity']) * 1000  # Convert g to kg and cc to l
    elif row['food_unit'] in ["cl"]:
        return (row['Quotazione Media'] / row['food_quantity']) * 100
    else:
        return None  # Return None for unsupported units

In [ ]:
# Apply the function to calculate the cost per kg or liter
final_prices_df['cost_per_kg_or_l'] = final_prices_df.apply(calculate_cost_per_kg_or_liter, axis=1)

# Display the final DataFrame with the new column
final_prices_df[["Food commodity ITEM_cons", "Final CF value", "Food commodity ITEM_wf", "Final WF value", "food_quantity", "food_unit", "cost_per_kg_or_l"]]

In [ ]:
final_prices_df.columns.tolist()

In [ ]:
# Keep only the relevant columns
final_prices_df = final_prices_df[["AGROVOC_label", "AGROVOC_uri", "FOOD COMMODITY GROUP_cons", "Mean", "Final CF value", "Final WF value", "cost_per_kg_or_l"]]

final_prices_df

In [ ]:
# Rename the columns for clarity
final_prices_df.columns = ["AGROVOC_label", "AGROVOC_uri", "food_typology", "Mean_consumption_italy", "Carbon Footprint (g CO2eq/g o cc)", "(Water Footprint liters) water/kg o liter", "cost_per_kg_or_l"]

In [ ]:
# Merging the three tomato rows into one by averaging the values
definitive_df = final_prices_df.groupby("AGROVOC_label").agg({"AGROVOC_uri": "first", "food_typology": "first", "Mean_consumption_italy": "first", "Carbon Footprint (g CO2eq/g o cc)": "mean", "(Water Footprint liters) water/kg o liter": "mean", "cost_per_kg_or_l": "first"}).reset_index()


In [ ]:
# Sorting the definitive DataFrame by Mean_consumption_italy
definitive_df = definitive_df.sort_values(by="Mean_consumption_italy", ascending=False).reset_index(drop=True)

definitive_df

In [ ]:
# Save the final DataFrame to a CSV file
definitive_df.to_csv("site/final_data/italy_food_data.csv", index=False, encoding="utf-8")

In [ ]:
# Assigning the correct data types to the first three columns
definitive_df["AGROVOC_label"] = definitive_df["AGROVOC_label"].astype(str)
definitive_df["AGROVOC_uri"] = definitive_df["AGROVOC_uri"].astype(str)
definitive_df["food_typology"] = definitive_df["food_typology"].astype(str)

print(definitive_df.dtypes)

In [ ]:
# Check correlation between carbon footprint and price
definitive_df_nona = definitive_df.dropna(subset=["Mean_consumption_italy", "Carbon Footprint (g CO2eq/g o cc)", "(Water Footprint liters) water/kg o liter", "cost_per_kg_or_l"])
definitive_df_nona

In [ ]:
definitive_df["AGROVOC_label"].to_list()

In [ ]:
numeric_df = definitive_df_nona.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numeric_df.corr()
correlation_matrix

In [ ]:
plt.scatter(numeric_df["Carbon Footprint (g CO2eq/g o cc)"], numeric_df["cost_per_kg_or_l"])
plt.title("cf vs cost per kg or liter")
plt.xlabel('cf')
plt.ylabel('cost per kg or liter')
plt.grid()
plt.show()

### Selecting other food for the menu and creating the dataframe with all the useful variables

In [ ]:
food_consumption = df_consumption.groupby("Exposure hierarchy (L5)", as_index=False)["Mean"].sum().sort_values('Mean', ascending=False)

In [ ]:
# Function to select rows based on a list of food items (even if they are not exact matches)
def select_rows_by_food_items(df, food_items):
    selected_rows = []
    for item in food_items:
        # Use regex to find rows that contain the food item (case-insensitive)
        pattern = re.compile(re.escape(item), re.IGNORECASE)
        matching_rows = df[df['Exposure hierarchy (L5)'].str.contains(pattern, na=False)]
        selected_rows.append(matching_rows)
    return pd.concat(selected_rows, ignore_index=True)

In [ ]:
list_food_items = ["yogurt", "ham", "tuna", "soya drink", "peas", "pears", "rice", "onion", "carrot", "zucch", "juice, orange", "salad", "eggplant", "orange", "banana", "salmon", "cod", "chocolate", "ice cream, milk-based"]

In [ ]:
rows_food = select_rows_by_food_items(food_consumption, list_food_items)

print(rows_food["Exposure hierarchy (L5)"].to_list())

In [ ]:
# Function to remove rows that contain specific substrings in a column
def remove_rows_with_substrings(df, column_name, substrings):
    for substring in substrings:
        pattern = re.compile(re.escape(substring), re.IGNORECASE)
        df = df[~df[column_name].str.contains(pattern, na=False)]
    return df

In [ ]:
food_rows = remove_rows_with_substrings(rows_food, "Exposure hierarchy (L5)", ["Broad", "Simple", "with pods", "French", "Fried", "Boiled", "Poached", "Common Quail", "cake", "chocolate sauce", "Couverture chocolate", "chocolate cake", "croissant", "chocolate spread", "Chocolate coated confectionery", "Cooked", "Ham, beef", "Bechamel", "Sandwich", "without pods", "Rice drink", "bread", "popped", "Liquorice", "Starch", "flour", "Smoked", "Canned", "Salted"])

# Remove duplicates based on the "Exposure hierarchy (L5)" column
food_rows = food_rows.drop_duplicates(subset=["Exposure hierarchy (L5)"])
# Reset the index of the DataFrame
food_rows = food_rows.reset_index(drop=True)

food_rows

In [ ]:
# Merging the chocolate rows and take the mean of the values
chocolate_rows = food_rows[food_rows["Exposure hierarchy (L5)"].str.contains("chocolate", case=False, na=False)]
chocolate_rows = chocolate_rows[~chocolate_rows["Exposure hierarchy (L5)"].str.contains("biscuit", case=False, na=False)]

# Remove attributes like Bitter, Milk, White, Dark, etc. from the chocolate rows
chocolate_rows["Exposure hierarchy (L5)"] = chocolate_rows["Exposure hierarchy (L5)"].str.replace(r'\b(bitter|milk|white|Filled|Gianduja)\b', '', regex=True, case=False).str.strip()

chocolate_rows = chocolate_rows.groupby("Exposure hierarchy (L5)", as_index=False).mean()

chocolate_rows

In [ ]:
# Remove rows containing "chocolate" but keep "Biscuits, chocolate"
food_rows = food_rows[~(food_rows["Exposure hierarchy (L5)"].str.contains("chocolate", case=False, na=False) & 
                        ~food_rows["Exposure hierarchy (L5)"].str.contains("Biscuits, chocolate", case=False, na=False))]

# Append the merged chocolate rows
food_rows = pd.concat([food_rows, chocolate_rows], ignore_index=True)

food_rows

In [ ]:
# Remove rows 3, 6, 10, 13
rows_to_remove = [3, 6, 10, 13]
food_rows = food_rows.drop(index=rows_to_remove).reset_index(drop=True)

food_rows

In [ ]:
# Enrich the food_rows DataFrame with AGROVOC labels and URIs
food_rows_enriched = enrich_with_agrovoc(food_rows, "Exposure hierarchy (L5)")

food_rows_enriched

In [ ]:
# Cleaning the labels in the food_rows DataFrame
food_rows['clean_label'] = food_rows["AGROVOC_label"].apply(normalize_label, language='en')

food_rows

In [ ]:
# Search for matches with the cf DataFrame using semantic matching
food_rows_matched = find_semantic_matches(
    source_df=food_rows,
    candidate_df=df_cf,
    source_column='clean_label',
    candidate_column='clean_label'
)

In [ ]:
food_rows_matched

In [ ]:
# Merge the food_rows DataFrame with the cF DataFrame on the matched food, keeping unmatched values as None
food_rows_merged = food_rows_matched.merge(df_cf, left_on='best_match', right_on='clean_label', how="left", suffixes=('_food', '_cf'))

In [ ]:
food_rows_merged

In [ ]:
# Manually fix the data
# Choose the correct ham row in the cF DataFrame
ham_rows = df_cf[df_cf["clean_label"].str.contains("ham", case=False, na=False)]

ham_row = ham_rows.iloc[1].to_frame().T

In [ ]:
# Choose the correct biscuit row in the cF DataFrame
biscuit_rows = df_cf[df_cf["clean_label"].str.contains("biscuit|cookie", case=False, na=False)]
biscuit_row = biscuit_rows.iloc[0].to_frame().T

In [ ]:
# Choose the correct chocolate row in the cF DataFrame
chocolate_rows = df_cf[df_cf["clean_label"].str.contains("dark chocolate|milk chocolate|white chocolate", case=False, na=False)]

chocolate_row = chocolate_rows.iloc[0].to_frame().T

In [ ]:
# Substitute the values in the food_rows_merged DataFrame for the ham, biscuit, and chocolate rows
substitute_part_rows(food_rows_merged, ham_row, "hake", "best_match", 10, df3=biscuit_row, value2="crispbread", df4=chocolate_row, value3="chocolate")

food_rows_merged

In [ ]:
# Remove duplicates. To choose the best row, we will keep the one without the (F) in the "Food commodity ITEM" column
def keep_best_rows(df, column_name):
    def pick_best(group, column_name=column_name):
        # Try to find a row without (F)
        non_f = group[~group[column_name].str.contains(r"\(F\)|\(fresh\)", na=False)]
        if not non_f.empty:
            return non_f.iloc[0]
        else:
            return group.iloc[0]
    return df.groupby("best_match", as_index=False, group_keys=False).apply(pick_best, include_groups = False).reset_index(drop=True)



In [ ]:
food_rows_merged = keep_best_rows(food_rows_merged, "Food commodity ITEM")

In [ ]:
food_rows_merged

In [ ]:
# Semantic matching with the water footprint DataFrame
food_rows_wf_matched = find_semantic_matches(
    source_df=food_rows_merged,
    candidate_df=df_wf,
    source_column='clean_label_food',
    candidate_column='clean_label'
)

In [ ]:
food_rows_wf_matched

In [ ]:
# Merge the merged DataFrame with the water footprint DataFrame on the matched food, keeping unmatched values as None
food_rows_wf_merged = food_rows_wf_matched.merge(df_wf, left_on='best_match', right_on='clean_label', how="left", suffixes=('_food', '_wf'))

In [ ]:
food_rows_wf_merged

In [ ]:
# Manually fix the data
# Choose the correct ham row in the wF DataFrame
ham_rows = df_wf[df_wf["clean_label"].str.contains("ham", case=False, na=False)]

ham_row = ham_rows.iloc[0].to_frame().T

In [ ]:
# Choose the correct biscuit row in the wF DataFrame
biscuit_rows = df_wf[df_wf["clean_label"].str.contains("biscuit|cookie", case=False, na=False)]
biscuit_row = biscuit_rows.iloc[0].to_frame().T

In [ ]:
# Choose the correct ice cream row in the wF DataFrame
ice_cream_rows = df_wf[df_wf["clean_label"].str.contains("ice cream", case=False, na=False)]

ice_cream_rows

In [ ]:
# Choose the correct tuna row in the wF DataFrame
tuna_rows = df_wf[df_wf["clean_label"].str.contains("tuna", case=False, na=False)]
tuna_rows

In [ ]:
# Substitute the values in the food_rows_merged DataFrame for the ham, biscuit, and chocolate rows
substitute_part_rows(food_rows_wf_merged, ham_row, "kephir", "best_match", 10, df3=biscuit_row, value2="crispbread")

food_rows_wf_merged

In [ ]:
food_rows_wf_merged = keep_best_rows(food_rows_wf_merged, "Food commodity ITEM_wf")

In [ ]:
# Since there are no tuna and no ice cream rows in the wF DataFrame, we will assign NaN to the last seven columns of the tuna and ice cream rows in the food_rows_wf_merged DataFrame
for idx, row in food_rows_wf_merged.iterrows():
    if "tuna" in row["Food commodity ITEM_food"].lower():
        food_rows_wf_merged.iloc[idx, -10:] = [np.nan] * 10  # Set the last seven columns to NaN
    elif "ice cream" in row["Food commodity ITEM_food"].lower():
        food_rows_wf_merged.iloc[idx, -10:] = [np.nan] * 10  # Set the last seven columns to NaN

In [ ]:
food_rows_wf_merged

In [ ]:
# Semantic matching with prices DataFrame
food_rows_prices_matched = find_semantic_matches(
    source_df=food_rows_wf_merged,
    candidate_df=prices_data,
    source_column='clean_label_food',
    candidate_column='clean_label'
)

In [ ]:
food_rows_prices_matched

In [ ]:
# Merge the food_prices_matched DataFrame with the prices DataFrame on the matched food, keeping unmatched values as None
food_rows_prices_merged = food_rows_prices_matched.merge(prices_data, left_on='best_match', right_on='clean_label', how="left", suffixes=('_food1', '_price'))

In [ ]:
food_rows_prices_merged

In [ ]:
# Manually fix the data
# Choose the correct ham row in the prices DataFrame
ham_rows = prices_data[prices_data["clean_label"].str.contains("prosciutto", case=False, na=False)]
ham_row = ham_rows.iloc[0].to_frame().T

# Choose the correct cod row in the prices DataFrame
cod_rows = prices_data[prices_data["clean_label"].str.contains("merluzzi", case=False, na=False)]
cod_row = cod_rows.iloc[0].to_frame().T

# Choose the correct orange row in the prices DataFrame
orange_rows = prices_data[prices_data["clean_label"].str.contains("arance", case=False, na=False)]
orange_row = orange_rows.iloc[0].to_frame().T

# Choose the correct rice row in the prices DataFrame
rice_rows = prices_data[prices_data["clean_label"].str.contains("riso", case=False, na=False)]
rice_row = rice_rows.iloc[0].to_frame().T

# Choose the correct soy milk row in the prices DataFrame
soy_milk_rows = prices_data[prices_data["clean_label"].str.contains("vegetali", case=False, na=False)]
soy_milk_row = soy_milk_rows.iloc[0].to_frame().T

# Choose the correct peas row in the prices DataFrame
peas_rows = prices_data[prices_data["clean_label"].str.contains("piselli", case=False, na=False)]
peas_row = peas_rows.iloc[0].to_frame().T

# Choose the correct pear row in the prices DataFrame
pear_rows = prices_data[prices_data["clean_label"].str.contains("pere", case=False, na=False)]
pear_row = pear_rows.iloc[0].to_frame().T

In [ ]:
# Substitute the values in the food_rows_prices_merged DataFrame for the ham, cod, orange, rice, and soy milk rows
substitute_part_rows(food_rows_prices_merged, ham_row, "hams", "AGROVOC_label", 6, df3=cod_row, value2="cod", df4=orange_row, value3="sweet oranges")
substitute_part_rows(food_rows_prices_merged, rice_row, "rice", "AGROVOC_label", 6, df3=soy_milk_row, value2="soy milk", df4=peas_row, value3="peas")
substitute_part_rows(food_rows_prices_merged, pear_row, "pears", "AGROVOC_label", 6)

In [ ]:
food_rows_prices_merged

In [ ]:
# Apply the helper function to the DataFrame
food_rows_prices_merged[["Final CF value", "Final WF value"]] = food_rows_prices_merged.apply(get_final_values, axis=1)

# Apply the function to extract food quantity and unit
food_rows_prices_merged['food_quantity'], food_rows_prices_merged['food_unit'] = zip(*food_rows_prices_merged['Descrizione Prodotto'].apply(extract_food_quantity))

# Apply the function to calculate the cost per kg or liter
food_rows_prices_merged['cost_per_kg_or_l'] = food_rows_prices_merged.apply(calculate_cost_per_kg_or_liter, axis=1)

# Keep only the relevant columns
food_rows_prices_merged = food_rows_prices_merged[["AGROVOC_label", "AGROVOC_uri", "FOOD COMMODITY GROUP_food", "Mean", "Final CF value", "Final WF value", "cost_per_kg_or_l"]]

# Rename the columns for clarity
food_rows_prices_merged.columns = ["AGROVOC_label", "AGROVOC_uri", "food_typology", "Mean_consumption_italy", "Carbon Footprint (g CO2eq/g o cc)", "(Water Footprint liters) water/kg o liter", "cost_per_kg_or_l"]

# Sorting the definitive DataFrame by Mean_consumption_italy
definitive_df_more_food = food_rows_prices_merged.sort_values(by="Mean_consumption_italy", ascending=False).reset_index(drop=True)

definitive_df_more_food

In [ ]:
# Concat the two definitive DataFrames
final_definitive_df = pd.concat([definitive_df, definitive_df_more_food], ignore_index=True)

In [ ]:
# Save the final DataFrame to a CSV file
final_definitive_df.to_csv("site/final_data/game_data.csv", index=False, encoding="utf-8")

In [ ]:
# Check correlation between carbon footprint and price
final_definitive_df_nona = final_definitive_df.dropna(subset=["Mean_consumption_italy", "Carbon Footprint (g CO2eq/g o cc)", "(Water Footprint liters) water/kg o liter", "cost_per_kg_or_l"])
final_definitive_df_nona

In [ ]:
# Correlation matrix
numeric_df_final = final_definitive_df_nona.select_dtypes(include=['float64', 'int64'])
correlation_matrix_final = numeric_df_final.corr()

correlation_matrix_final